# Quickstart Sinabs
If you're familiar with how SNNs work, you might find this quick overview about *Sinabs* useful.

## Sinabs is based on PyTorch
All of Sinabs' layers inherit from `torch.nn.Module`. Thus you will be able to access your parameters, wrap layers in a `nn.Sequential` module and all the other things that you would do with a normal PyTorch layer. 

## How to define your network
We want to re-use as much PyTorch functionality as possible. We use Linear, Conv2d and AvgPool layers to define weight matrices, whereas *Sinabs* layers add state as well as the non-linear activation to each of those weight layers. This is a definition of a simple SNN which takes as an input a tensor of (Batch, Time, Channels):


In [ ]:
import torch
import torch.nn as nn
import sinabs.layers as sl

model = nn.Sequential(
    nn.Linear(16, 32),
    sl.LIF(tau_mem=20.),
    nn.Linear(32, 4),
    sl.LIF(tau_mem=40.),
)

# (Batch, Time, Channels)
input = torch.rand(8, 100, 16)

When working with convolutional connectivity, a `nn.Conv2d` layer only takes as input a tensor of (Batch, Channels, Height, Width). If we feed a tensor that has an additional time dimension (Batch, Time, Channels, Height, Width) to such a layer, we will receive an error. In order for us to apply 2D convolutions across time, we have to make use of a small trick where we flatten batch and time dimension before feeding it to the Conv layer. If the input is flattened, we will need to unflatten it for each Sinabs layer by using `Squeeze` versions of layers.

In [ ]:
batch_size = 8

conv_model = nn.Sequential(
    nn.Conv2d(2, 16, kernel_size=3),
    sl.LIFSqueeze(tau_mem=20., batch_size=batch_size),
    nn.Conv2d(16, 32, kernel_size=3),
    sl.LIFSqueeze(tau_mem=20., batch_size=batch_size),
    nn.Flatten(),
    nn.Linear(32, 4),
)

# (Batch*Time, Channels, Height, Width)
input = torch.rand()